# BipartitePandas example

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# Add pytwoway to system path, do not run this
import sys
sys.path.append('../../..')

In [ ]:
# Import the pytwoway package 
# (Make sure you have installed it using pip install pytwoway)
import pytwoway as tw

## Simulate some data

The package contains functions to simulate data. We use this here to keep things simple.

In [ ]:
# For the example, we simulate data
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
display(sim_data)

## BipartiteData

BipartiteData is a superclass with 3 subclasses:
- BipartiteLong
- BipartiteLongCollapsed
- BipartiteEventStudy

The user interfaces with BipartiteData which converts between the 3 subclasses.

In [ ]:
# Our data is in long form (each row gives a single observation)
columns_req = ['wid', 'comp', 'fid', 'year']
columns_opt = []
reference_dict = {'wid': 'wid', 'comp': 'comp', 'fid': 'fid', 'year': 'year'}
bd = tw.BipartiteBase(sim_data, columns_req=columns_req, columns_opt=columns_opt, reference_dict=reference_dict) # Long is default formatting, most data is long
display(bd)

In [ ]:
print(type(bd))
print(type(bd.copy()))
print(bd.copy().connected)

In [ ]:
print('j:', bd.col_included('j'))
print('wid:', bd.col_included('wid'))
print('fid:', bd.col_included('fid'))

In [ ]:
bd2 = tw.BipartiteBase(bd)

In [ ]:
print('j:', bd2.col_included('j'))
print('wid:', bd2.col_included('wid'))
print('fid:', bd2.col_included('fid'))

In [ ]:
bd2['a'] = 1
bd2

In [ ]:
bd2.drop('a')

In [ ]:
bd.clean_data() # Should always clean data after initializing
# Let's see how our data looks
display(bd)

## Converting formats

In [ ]:
# While our original data is long, we might want it to be in event study form (each row gives two consecutive observations)
bd = bd.get_es()
display(bd)

In [ ]:
# We can also use event study data to retrive cross section data (cs=1 gives y1 as y1 for both stayers and movers; cs=0 gives y2 as y1 for only movers - this allows (almost) all income data to be accessed from the y1 column. Note that for movers, the last observation for each worker is not available without manipulation as it is shifted to the y2 column. Also note that the y1 row contains duplicates for all mover income, except for the first period.)
display(bd.get_cs())

In [ ]:
# Maybe we want to convert back into long form
bd = bd.get_long()
display(bd)

In [ ]:
# Now suppose we want to collapse by employment spells (so any consecutive observations with the same worker in the same firm are collapsed into 1 observation)
bd = bd.get_collapsed_long()
display(bd)

In [ ]:
# We can then check out the event study using collapsed data
bd = bd.get_es()
display(bd)

In [ ]:
# We can then go back to collapsed long
bd.get_collapsed_long()
display(bd)

## Clustering

In [ ]:
# Starting over
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
display(bd)

In [ ]:
# We can cluster from any format and clusters stay when reformatting
bd.cluster()
display(bd)

In [ ]:
bd = bd.get_es()
display(bd)

In [ ]:
bd = bd.get_long()
bd = bd.get_collapsed_long()
display(bd)

In [ ]:
bd = bd.get_es()
display(bd)

## Clustering options

In [ ]:
# We can cluster on a specific year in long form
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
bd.cluster({'year': 1})
display(bd)

In [ ]:
# We can cluster on movers or stayers in any form
# Note that not all firms are clustered, so some rows have NaNs
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
bd.cluster({'stayers_movers': 'stayers'})
display(bd)

In [ ]:
# We can cluster on movers or stayers in any form
# Set 'dropna': True if you want to drop firms that don't get clustered (note that this fully resets firm and worker ids, since they must be contiguous for the FE/CRE/BLM estimators to work)
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
bd.cluster({'stayers_movers': 'stayers', 'dropna': True})
display(bd)

In [ ]:
# We can cluster on movers or stayers in any form
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
bd.cluster({'stayers_movers': 'movers'})
display(bd)

## Some extra features

In [ ]:
# # We can get some summary statistics by printing the BipartiteData object
# sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
# sim_data = sim_data.rename({'fid': 'f1i'}, axis=1)
# bd = tw.BipartitePandas(sim_data, col_dict={'fid': 'f1i'})
# bd.clean_data()
# display(bd)

In [ ]:
# BipartiteData does some nice column inference
# Suppose some of our columns are mislabeled - create a col_dict to clarify which columns to correct
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data = sim_data.rename({'fid': 'f1i'}, axis=1)
bd = tw.BipartitePandas(sim_data, col_dict={'fid': 'f1i'})
bd.clean_data()
display(bd)

In [ ]:
# Suppose we have already clustered - the class determines this automatically
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data['j'] = 1
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
display(bd)

In [ ]:
# Now suppose we mislabeled our clusters - the class drops this column
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data['j1'] = 1
bd = tw.BipartitePandas(sim_data)
bd.clean_data()
display(bd)

In [ ]:
# We can make sure our columns are included by specifying the corrected labels in col_dict (note that if the variable of interest requires 2 columns, e.g. weight 1 and weight 2 for event study data, BOTH columns are required - if only one column is included, it is automatically dropped.)
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data['j1'] = 1
bd = tw.BipartitePandas(sim_data, col_dict={'j': 'j1'})
bd.clean_data()
display(bd)

In [ ]:
# We may also specify a col_dict that overwrites a column label that would normally be inferred, and inference is skipped
sim_data = tw.SimTwoWay().sim_network()[['wid', 'fid', 'comp', 'year']]
sim_data = sim_data.rename({'fid': 'j'}, axis=1)
bd = tw.BipartitePandas(sim_data, col_dict={'fid': 'j'})
# Note that even though we have a column named j, because we specify it is the 'fid' column it is not inferred as the cluster values
bd.clean_data()
display(bd)